In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import string
import csv
import shutil
import copy
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import spacy
from spacy import lookups
from spacy import tokenizer
import nltk
from nltk import wordpunct_tokenize
from nltk import word_tokenize
from nltk import pos_tag
nlp = spacy.load("en_core_web_sm")
from nltk import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize as st
from nltk.stem import WordNetLemmatizer as wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import gensim.models
import pickle
import joblib

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


[nltk_data] Downloading package stopwords to /Users/Alice/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import string
import csv
import spacy
from spacy import lookups
from spacy import tokenizer
import re
import nltk
from nltk import wordpunct_tokenize
from nltk import word_tokenize
from nltk import pos_tag
nlp = spacy.load("en_core_web_sm")

In [3]:
# Import training data 
allData = pd.read_csv("MN_48_152_2 19 2024 .txt", delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

In [4]:
# Text preprocessing and adds speaker column
speakers = []
for i in range(len(allData)):
    if (':' in allData.iloc[i, 0]):
        index = allData.iloc[i, 0].index(':')
        speakers.append((allData.iloc[i, 0])[:index+1])
        allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0]) 
    else:
        speakers.append('F:')
    allData.iloc[i, 0] = re.sub("[a-zA-Z]:\s+", "", allData.iloc[i, 0])
allData.insert (0, "speaker", speakers)

In [5]:
class Sentence:
# Creates a Sentence object
# text = text of Sentence
# multi = which occurence of negation is being looked at in the sentence
  # eg. if the variable be is set to 2, the model will observe whether the 2nd negator is part of a multiple negator
    # pair in the sentence
# num = the index of the Sentence in the original csv
    multNeg = 0
    r1 = 0; r2 = 0;
    def __init__(self, text, multi, num, true):
        self.text = text
        self.num = num
        self.multi = multi
        self.true = true

In [6]:
# Preproccesses training text for linguistic analysis

lines = []

with open("MN_48_152_2 19 2024 .txt", "r", encoding="utf-8") as tsv_file: #these files are tab separated but you can easily change this to CSV
    tsv_reader = csv.reader(tsv_file, delimiter="\t")
    header = next(tsv_reader)
    for row in tsv_reader:
        row = row[:1]
        row[0] = re.sub("[a-zA-Z]:\s+", "", row[0]) #removes the interviewer tag
        row[0] = re.sub("\s{2,}", " ", row[0]) #removes excessive spaces
        row[0] = re.sub("’|‘", "'", row[0]) #fixes apostrophes
        row[0] = re.sub("—", "--", row[0]) #m-dash was causing formatting issues, changed it to two dashes
        row[0] = re.sub("“", '"', row[0]) #fixes quotation marks    
        lines.append(row[0])
         
print(len(lines))

155


In [7]:
# Sets list of negators
negatives = ["no", "nothing", "never", "nobody", "nowhere", "neither", "n't", "n’t", "not", "nor", "none", "nothin’","nothin'", "ain’t", "ain't"] 

In [8]:
# Create an array of Sentence objects with at least one negator

sen = []
for line in range(len(lines)):
        multi = 1
        numNeg = 0
        parsed = nlp(lines[line])
        for i, word in enumerate(parsed):
            if (word.text.lower() in negatives):
                numNeg += 1
        if (numNeg == 1):
            sen.append(Sentence(lines[line], multi, line, 0))
        elif (numNeg > 1):
            sen.append(Sentence(lines[line], multi, line, 1))
        while (numNeg > 1):
            multi += 1
            sen.append(Sentence(lines[line], multi, line, 1))
            numNeg -= 1
for Sent in range(len(sen)):
    sen[Sent].text = (sen[Sent].text).replace('"','')

In [9]:
# # Creates empty arrays corresponding to the parts of speech of the words including and surrounding 'be'
# E.g. w = word, p = previous, pp = preprevious, a = after, pa = postafter, etc

w = []
ppp = []
pp = []
p = []
a = []
pa = []
ppa = []

In [10]:
# Tags the Sentence objects with each rule

index = -1
senNum = 1
numNeg = 0
negIndices = []

for Sentence in sen:
    index += 1 
    numNeg = 0
    if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
        senNum += 1
    else:
        senNum = 1
    parsed = nlp(Sentence.text)
# Find an instance of negation
    for i, word in enumerate(parsed):
        if (i >= 0 and (word.text.lower() in negatives) and Sentence.true == 1 and Sentence.multNeg != 1):
            numNeg += 1
            if (numNeg == senNum):
                w.append(word.pos_)
                negIndices.append(i)
                negChildren = list(word.children)
                if(word.head == word):
                    negSibling = []
                else:
                    negSibling = list(word.head.children)
                negDep = word.dep_
                negPOS = word.pos_
                negIndex = i
                num = 1
                next = None
                prev = None
# Progress through the sentences forwards, if another instance of negation is found within the clause, tag sentence with rule 1 and set multNeg to 1
                while ((i+num < len(parsed)) and (num < 9) and (parsed[i+num].pos_ != "PUNCT" and parsed[i+num].pos_ != "CCONJ" and parsed[i+num].pos_ != "SCONJ" and parsed[i+num].text != "–")):# and (parsed[i+num].pos_ != 'PRON' and parsed[i+num].pos_ != 'PROP' and parsed[i+num].pos_ != 'NOUN') or parsed[i+num].text in negatives)): 
                    next = parsed[i+num]
                    if next.text.lower() in negatives and (word.text.lower() != "no" or next.text.lower() != "no"):
                        Sentence.r1 = 1
                        Sentence.multNeg = 1
                        print(num)
                        print(Sentence.text)
                    num += 1
                if (Sentence.r1 != 1):
                    Sentence.r1 = 0
                num = 1
# Progress through the sentences backwards, if another instance of negation is found within the clause, tag sentence with rule 2 and set multNeg to 1
                while ((i-num >= 0) and (num < 9) and (parsed[i-num].pos_ != "PUNCT") and parsed[i-num].pos_ != "CCONJ" and parsed[i-num].pos_ != "SCONJ" and parsed[i-num].text != "–"):# and ((parsed[i-num].pos_ != 'PRON' and parsed[i-num].pos_ != 'PROP' and parsed[i-num].pos_ != 'NOUN') or parsed[i-num].text in negatives): 
                    prev = parsed[i-num]
                    if prev.text.lower() in negatives and (word.text.lower() != "no" or prev.text.lower() != "no"):
                        Sentence.r2 = 1
                        Sentence.multNeg = 1
                        print(num)
                        print(Sentence.text)
                    num += 1
                if (Sentence.r2 != 1):
                    Sentence.r2 = 0
# If neither rule 1 or rule 2 apply, set Sentence.multNeg to 0 to indicate the lack of multiple negation
                if (Sentence.r1 !=1 and Sentence.r2 != 1):
                    Sentence.multNeg = 0

2
Now; you're not getting no spanking now.
2
Now; you're not getting no spanking now.
3
No you can't mess up anything.
3
No you can't mess up anything.
2
I've been all up there over there but I ain't been nowhere in Florida.
2
I've been all up there over there but I ain't been nowhere in Florida.
5
But these children; if they get to schooling; they won't have to work in nobody's field.
5
But these children; if they get to schooling; they won't have to work in nobody's field.
2
Didn't have no radio home.
2
Didn't have no radio home.
1
I don't never want to see you like that.
1
I don't never want to see you like that.
1
But that wasn't nothing for me to think that he was doing that to me; because of the black or white thing.
1
But that wasn't nothing for me to think that he was doing that to me; because of the black or white thing.
1
But I would tell them to trust in God; be sure to trust in God; don't never forget wherever you go to make sure that you put your trust in God and to do the

In [11]:
# Consolidation pna values of duplicated sentences
# e.g. if any have multiple negation, assign all mult neg values to 1, else keep them as 0

testError = []
multNeginSentence = []

for i in range(len(sen)-1):
    if (i < len(sen)-1):
        j = i+1 
        if (sen[i].text != sen[j].text):
            if (sen[i].multNeg == 1):
                multNeginSentence.append(True)
            else:
                multNeginSentence.append(False)
        while (sen[i].text == sen[j].text):
            if (sen[i].multNeg == 1 or sen[j].multNeg) == 1:
                multNeginSentence.append(True)
            else:
                multNeginSentence.append(False)
            if (sen[j].multNeg == 1):
                sen[i].multNeg == sen[j].multNeg
            if (sen[i].multNeg == 1):
                sen[j].multNeg == sen[i].multNeg
            testError.append((j, sen[j]))
            if (j < len(sen)-1):
                j +=1
            else:
                break


if len(multNeginSentence) != len(sen):
    if (sen[-1].multNeg == 1):
        multNeginSentence.append(True)
    else:
        multNeginSentence.append(False)


In [12]:
# Goes back to the original data and annotates each sentences with multiple negation

multNeg = []
for i in range(len(allData)):
    m = 0
    for j in range(len(sen)):
        if i == sen[j].num:
            if (m != 1):
                m = sen[j].multNeg
            if (m == 1 and ("%MN" not in allData.loc[i, 'speaker'])):
                allData.loc[i, 'speaker'] = allData.loc[i, 'speaker'] + " %MN"
    multNeg.append(m)
print(len(multNeg))
print(len(allData))
allData['Multiple Negators'] = multNeg

155
155


In [13]:
# Print classification report

predictions = allData['Multiple Negators']
y = allData['Multiple Negation']
target_names = ['agreement', 'disagreement']
print(classification_report(y, predictions, target_names=target_names))

              precision    recall  f1-score   support

   agreement       1.00      0.99      1.00       117
disagreement       0.97      1.00      0.99        38

    accuracy                           0.99       155
   macro avg       0.99      1.00      0.99       155
weighted avg       0.99      0.99      0.99       155



In [14]:
# Write annotations to txt file

with open('multNeg.txt', 'w+', newline='') as file:
   allData.to_csv('multNeg.txt', sep = '\t')